# 1. 背景：为什么用 MSE Loss？

在回归问题（预测一个连续值，比如房价、气温）中，我们希望预测值 y^尽可能接近真实值 𝑦

- 均方误差 (`Mean Squared Error`, `MSE`) 是最常见的度量方法：

    - 计算预测值和真实值的差（误差）；
    
    - 平方后取平均，保证误差为正，且放大大误差。

# 2. 定义

In [ ]:
torch.nn.MSELoss(
    size_average=None,
    reduce=None,
    reduction='mean'
)


| 参数                         | 作用                                                                                       |
| -------------------------- | ---------------------------------------------------------------------------------------- |
| **reduction**              | 控制损失的聚合方式：<br> - `'mean'`（默认）：求所有样本的平均损失<br> - `'sum'`：把所有样本损失相加<br> - `'none'`：逐元素返回损失值 |
| **size\_average / reduce** | 已废弃，统一用 `reduction` 参数控制。                                                                |


## 使用示例
### (1) 单一样本

In [1]:
import torch
import torch.nn as nn

loss_fn = nn.MSELoss()

# 预测值和真实值
y_pred = torch.tensor([2.5])
y_true = torch.tensor([3.0])

loss = loss_fn(y_pred, y_true)
print(loss.item())  # 输出 0.25 = (2.5 - 3.0)^2


0.25


### (2) 多个样本

In [ ]:
loss_fn = nn.MSELoss()

y_pred = torch.tensor([2.5, 0.0, 2.1, 7.8])
y_true = torch.tensor([3.0, -0.5, 2.0, 7.0])

loss = loss_fn(y_pred, y_true)
print(loss.item())
# 计算过程:
# ((2.5-3.0)^2 + (0.0-(-0.5))^2 + (2.1-2.0)^2 + (7.8-7.0)^2) / 4
# = (0.25 + 0.25 + 0.01 + 0.64) / 4 = 0.2875


### (3) 控制 reduction

In [ ]:
loss_fn_sum = nn.MSELoss(reduction='sum')
loss_fn_none = nn.MSELoss(reduction='none')

y_pred = torch.tensor([1.0, 2.0, 3.0])
y_true = torch.tensor([2.0, 2.0, 2.0])

print(loss_fn_sum(y_pred, y_true))   # 2.0 (平方和)
print(loss_fn_none(y_pred, y_true))  # tensor([1., 0., 1.]) (逐元素损失)


## 特点

✅ 优点：

- 简单直观，常用于回归。

- 对较大误差更敏感（因为平方）。

⚠️ 缺点：

- 对异常值（`outliers`）非常敏感（大误差平方后会主导损失）。

- 有时会收敛慢，可以考虑 `MA`（`L1Loss`） 或 `HuberLoss`。

---
## 应用场景

- 回归问题（房价预测、股票预测）

- 自编码器（重建误差）

- GAN 判别器的某些变体

- 强化学习中的 Q 值回归

# 📊 损失函数对比表

| 损失函数             | 特点          | 优点                    | 缺点                         | 适用场景                  |                          |                  |
| ---------------- | ----------- | --------------------- | -------------------------- | --------------------- | ------------------------ | ---------------- |
| **MSE (均方误差)**   | 误差平方后再平均    | - 对大误差敏感<br>- 常见且易于优化 | - 对异常值非常敏感（outliers 会主导损失） | 普通回归、图像重建、自编码器        |                          |                  |
| **MAE (平均绝对误差)** | 直接取绝对值                     | - 对异常值鲁棒<br>- 反映中位数趋势 | - 不可导点在 0 处<br>- 收敛速度可能慢 | 对异常值敏感的数据（金融、医疗） |
| **Huber Loss**   | 结合 MSE（对小误差）和 MAE（对大误差）| - 对异常值不敏感（比MSE稳健）<br>- 处处可导（比MAE利于优化）<br>- 平衡精度与鲁棒性| - 需手动调参（δ 阈值）<br>- 计算稍复杂于 MSE/MAE|- 回归任务含少量异常值<br>- 需兼顾准确性和鲁棒性（如目标检测、金融预测、机器人控制）|                          |                  |


- 直观理解

    - MSE：大误差会被放大 🚨
    - MAE：所有误差一视同仁 ⚖️
    - Huber：小误差用平方（收敛快），大误差用绝对值（抗异常） 🔧